In [ ]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    image_size = 128  # the generated image resolution
    train_batch_size = 16
    eval_batch_size = 16  # how many images to sample during evaluation
    num_epochs = 5000
    gradient_accumulation_steps = 1
    learning_rate = 1e-3
    lr_warmup_steps = 500
    save_image_epochs = 10
    save_model_epochs = 30
    mixed_precision = "no"  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = "ddpm-butterflies-128"  # the model name locally and on the HF Hub

    push_to_hub = False  # whether to upload the saved model to the HF Hub
    hub_model_id = "<your-username>/<my-awesome-model>"  # the name of the repository to create on the HF Hub
    hub_private_repo = False
    overwrite_output_dir = True  # overwrite the old model when re-running the notebook
    seed = 0


config = TrainingConfig()

## Dataset

In [ ]:
# from datasets import load_dataset

# config.dataset_name = "huggan/smithsonian_butterflies_subset"
# dataset = load_dataset(config.dataset_name, split="train")

# from torchvision import transforms

# preprocess = transforms.Compose(
#     [
#         transforms.Resize((config.image_size, config.image_size)),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.5], [0.5]),
#     ]
# )

# def transform(examples):
#     images = [preprocess(image.convert("RGB")) for image in examples["image"]]
#     return {"images": images}


# dataset.set_transform(transform)

# import torch

# train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=config.train_batch_size, shuffle=True)

In [ ]:
from open_anything_diffusion.datasets.synthetic_dataset import SyntheticDataModule
dataset= SyntheticDataModule(
    batch_size=1,
    seed=42
)

In [ ]:
single_train_dataloader = list(enumerate(dataset.train_dataloader()))[:1]
single_val_dataloader = single_train_dataloader
train_dataloader = dataset.train_dataloader()
val_dataloader = dataset.val_dataloader(bsz=1)

## Visual

In [ ]:
# import matplotlib.pyplot as plt

# fig, axs = plt.subplots(1, 4, figsize=(16, 4))
# for i, image in enumerate(dataset[:4]["image"]):
#     axs[i].imshow(image)
#     axs[i].set_axis_off()
# fig.show()

In [ ]:
sample = list(enumerate(train_dataloader))[0][1]
sample.delta.shape

In [ ]:
sample.batch

In [ ]:
import torch
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
from open_anything_diffusion.metrics.trajectory import normalize_trajectory

sample = list(enumerate(train_dataloader))[1][1]
pcd = sample.pos.cpu().numpy()
flow = normalize_trajectory(sample.delta)[:, 0, :]

animation = FlowNetAnimation()


animation.add_trace(
    torch.as_tensor(pcd),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([pcd]),
    torch.as_tensor([flow.squeeze().cpu().numpy()]),
    "red",
)

In [ ]:
fig = animation.animate()
fig.show()

## Model

In [ ]:
# from diffusers import UNet2DModel

# model = UNet2DModel(
#     sample_size=config.image_size,  # the target image resolution
#     in_channels=3,  # the number of input channels, 3 for RGB images
#     out_channels=3,  # the number of output channels
#     layers_per_block=2,  # how many ResNet layers to use per UNet block
#     block_out_channels=(128, 128, 256, 256, 512, 512),  # the number of output channels for each UNet block
#     down_block_types=(
#         "DownBlock2D",  # a regular ResNet downsampling block
#         "DownBlock2D",
#         "DownBlock2D",
#         "DownBlock2D",
#         "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
#         "DownBlock2D",
#     ),
#     up_block_types=(
#         "UpBlock2D",  # a regular ResNet upsampling block
#         "AttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
#         "UpBlock2D",
#         "UpBlock2D",
#         "UpBlock2D",
#         "UpBlock2D",
#     ),
# )

In [ ]:
# sample_image = dataset[0]["images"].unsqueeze(0)
# print("Input shape:", sample_image.shape)

# print("Output shape:", model(sample_image, timestep=0).sample.shape)

In [ ]:
import rpad.pyg.nets.pointnet2 as pnp
from diffusers.models.embeddings import TimestepEmbedding, Timesteps
class PNDiffuser(torch.nn.Module):
    def __init__(self):
        super(PNDiffuser, self).__init__()
        self.predictor = pnp.PN2Dense(
            # in_channels = self.in_channels + timestep_input_dim,
            in_channels= 3 + 64,
            out_channels=3 * 1,
            p=pnp.PN2DenseParams(),
        )

        self.sample_size=1200
        self.time_proj = Timesteps(64, True, 0)
        timestep_input_dim = 64
        self.time_embedding = TimestepEmbedding(timestep_input_dim, 64)

    def forward(
        self,
        context,
        noisy_input,
        timestep,
    ):
        # time embedding
        timesteps = timestep
        if not torch.is_tensor(timesteps):
            timesteps = torch.tensor(
                [timesteps], dtype=torch.long, device=noisy_input.device
            )
        elif torch.is_tensor(timesteps) and len(timesteps.shape) == 0:
            timesteps = timesteps[None].to(noisy_input.device)

        # broadcast to batch dimension in a way that's compatible with ONNX/Core ML
        timesteps = timesteps * torch.ones(
            noisy_input.shape[0], dtype=timesteps.dtype, device=timesteps.device
        )
    
        t_emb = self.time_proj(timesteps).float()
        t_emb = self.time_embedding(t_emb)
        t_emb = t_emb.unsqueeze(-1).repeat(1, 1, self.sample_size)


        context = context.to(noisy_input.device)
        # print(torch.flatten(noisy_input, start_dim=1, end_dim=2).shape, t_emb.shape)
        context.x = torch.cat(
            (torch.flatten(noisy_input, start_dim=1, end_dim=2), t_emb), dim=1
        )  # (B, 3 + 64 , N)
        # breakpoint()
        context.x = torch.flatten(context.x.permute(0, 2, 1), start_dim=0, end_dim=1)
        x = self.predictor(context)
        x = x.reshape(-1, self.sample_size, 3, 1).permute(0, 2, 3, 1)

        # print(x.shape, timesteps.shape)
        x = x / timesteps
        return x

model = PNDiffuser()

## Train

#### Scheduler

In [ ]:
from diffusers import DDPMScheduler
noise_scheduler = DDPMScheduler(num_train_timesteps=100)

from diffusers.optimization import get_cosine_schedule_with_warmup

optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(train_dataloader) * config.num_epochs),
)

In [ ]:
# import torch
# from PIL import Image
# from diffusers import DDPMScheduler

# noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
# noise = torch.randn(sample.delta.shape)
# timesteps = torch.LongTensor([50])
# noisy_image = noise_scheduler.add_noise(sample.delta, noise, timesteps)

# # Image.fromarray(((noisy_image.permute(0, 2, 3, 1) + 1.0) * 127.5).type(torch.uint8).numpy()[0])

In [ ]:
sample = list(enumerate(train_dataloader))[0][1]
pcd = sample.pos.cpu().numpy()
flow = normalize_trajectory(sample.delta)[:, 0, :]

noise = torch.randn(sample.delta.shape)
timesteps = torch.LongTensor([50])
noisy_flow = noise_scheduler.add_noise(sample.delta, noise, timesteps)


animation = FlowNetAnimation()

animation.add_trace(
    torch.as_tensor(pcd),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([pcd]),
    torch.as_tensor([noisy_flow.squeeze().cpu().numpy()]),
    "red",
)

fig = animation.animate()
fig.show()

In [ ]:
import torch.nn.functional as F

for step, batch in enumerate(train_dataloader):
    noise_pred = model(batch, noisy_flow.unsqueeze(0).permute(0, 2, 3, 1), timesteps)
    break

# print(noise_pred.squeeze(0).permute(2, 1, 0).shape, noise.shape)
loss = F.mse_loss(noise_pred.squeeze(0).permute(2, 1, 0), noise)
print(loss)

In [ ]:
# from diffusers import DDPMPipeline
# # from diffusers.utils import make_image_grid
# import os

# animations = []

# def evaluate(config, epoch, pipeline):
#     # Sample some images from random noise (this is the backward diffusion process).
#     # The default pipeline output type is `List[PIL.Image]`
#     images = pipeline(
#         batch_size=config.eval_batch_size,
#         generator=torch.manual_seed(config.seed),
#     ).images

#     # # Make a grid out of the images
#     # image_grid = make_image_grid(images, rows=4, cols=4)

#     # # Save the images
#     # test_dir = os.path.join(config.output_dir, "samples")
#     # os.makedirs(test_dir, exist_ok=True)
#     # image_grid.save(f"{test_dir}/{epoch:04d}.png")

In [ ]:
from accelerate import Accelerator
from huggingface_hub import create_repo, upload_folder
from tqdm.auto import tqdm
from pathlib import Path
import os

def train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    model.train()
    # Initialize accelerator and tensorboard logging
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs"),
    )
    if accelerator.is_main_process:
        if config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)
        if config.push_to_hub:
            repo_id = create_repo(
                repo_id=config.hub_model_id or Path(config.output_dir).name, exist_ok=True
            ).repo_id
        accelerator.init_trackers("train_example")

    # Prepare everything
    # There is no specific order to remember, you just need to unpack the
    # objects in the same order you gave them to the prepare method.
    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, lr_scheduler
    )

    global_step = 0

    # Now you train the model
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            clean_pcd = batch.pos
            clean_flow = batch.delta
            # Sample noise to add to the images
            noise = torch.randn(clean_flow.shape, device=clean_flow.device)
            bs = 1

            # Sample a random timestep for each image
            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps, (bs,), device=clean_flow.device,
                dtype=torch.int64
            )

            # Add noise to the clean images according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_flow = noise_scheduler.add_noise(clean_flow, noise, timesteps)

            with accelerator.accumulate(model):
                # Predict the noise residual
                noise_pred = model(batch, noisy_flow.unsqueeze(0).permute(0, 2, 3, 1), timesteps)
                # print(noise_pred.shape, noise.shape)
                loss = F.mse_loss(noise_pred.squeeze(0).permute(2, 1, 0), noise)
                # print(loss)
                accelerator.backward(loss)

                accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

        # # After each epoch you optionally sample some demo images with evaluate() and save the model
        # if accelerator.is_main_process:
        #     pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)

        #     # if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:
        #     #     evaluate(config, epoch, pipeline)

        #     if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
        #         if config.push_to_hub:
        #             upload_folder(
        #                 repo_id=repo_id,
        #                 folder_path=config.output_dir,
        #                 commit_message=f"Epoch {epoch}",
        #                 ignore_patterns=["step_*", "epoch_*"],
        #             )
        #         else:
        #             pipeline.save_pretrained(config.output_dir)

In [ ]:
from accelerate import notebook_launcher

args = (config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler)

notebook_launcher(train_loop, args, num_processes=1)

## Eval

In [ ]:
model.eval()
for id, sample in enumerate(val_dataloader):
    # breakpoint()
    batch_size = sample.pos.shape[0] // 1200
    noisy_input = (
        torch.randn_like(sample.delta)
        .float()
        .to('cuda')
    )

    animation = FlowNetAnimation()

    for t in noise_scheduler.timesteps:
        # model_output = self.model(
        #     noisy_input, encoder_hidden_states=condition, timestep=t
        # ).sample
        # breakpoint()
        model_output = model(sample, noisy_input.unsqueeze(0).permute(0, 2, 3, 1), t)
        print(t)

        noisy_input = noise_scheduler.step(
            model_output.squeeze(0).permute(2, 1, 0), t, noisy_input
        ).prev_sample

        # print(noisy_input.shape)
        if t % 5 == 0 or t == 99:
            flow = (
                # torch.nn.functional.normalize(noisy_input, p=2, dim=1)
                # .squeeze()
                # .permute(1, 0)
                normalize_trajectory(
                    noisy_input,
                )[:, 0, :]
            )
            animation.add_trace(
                torch.as_tensor(pcd),
                torch.as_tensor(
                    [pcd]
                ),
                torch.as_tensor(
                    [
                        flow
                        .detach()
                        .cpu()
                        .numpy()
                    ]
                ),
                "red",
            )

        # print(model_output)
        # print(noisy_input)

    break

In [ ]:
print(noisy_input)

In [ ]:
fig = animation.animate()
fig.show()